In [1]:
import os
import json
import hydra
import torch
import shutil
import warnings
import statistics
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from torch.optim.lr_scheduler import MultiStepLR
from torch_geometric.data import Dataset, DataLoader
from dataset import get_dataset, get_dataloader, get_concept_dataloader

from deepchem.models.torch_models import MPNNModel
from deepchem.metrics import accuracy_score, roc_auc_score
import deepchem as dc
import dgl

from rdkit import Chem
from rdkit.Chem import Draw, MACCSkeys, AllChem, Descriptors, rdmolfiles
from rdkit.Chem.Descriptors import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

from sklearn.model_selection import train_test_split

/home/michelaproietti/.conda/envs/thesis/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path="config", job_name="test_app")
cfg = compose(config_name="config", overrides=[])
print(OmegaConf.to_yaml(cfg))

models:
  gnn_saving_dir: ''
  gnn_name: gcn
  n_heads: 1
  param:
    hiv:
      learning_rate: 0.001
      weight_decay: 0.0005
      milestones: None
      gamma: None
      batch_size: 64
      num_epochs: 200
      num_early_stop: 20
      gnn_latent_dim:
      - 128
      - 128
      - 128
      gnn_dropout: 0.0
      add_self_loop: true
      gcn_adj_normalization: true
      gnn_emb_normalization: false
      graph_classification: true
      node_classification: false
      gnn_nonlinear: relu
      readout: sum
      fc_latent_dim:
      - 128
      fc_dropout: 0.0
      fc_nonlinear: relu
    clintox:
      learning_rate: 0.01
      weight_decay: 0.0
      milestones: None
      gamma: None
      batch_size: 128
      num_epochs: 400
      num_early_stop: 20
      gnn_latent_dim:
      - 128
      - 128
      - 128
      gnn_dropout: 0.0
      add_self_loop: true
      gcn_adj_normalization: false
      gnn_emb_normalization: false
      graph_classification: true
      node_

/home/michelaproietti/.conda/envs/thesis/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [3]:
config = cfg

if torch.cuda.is_available():
    device = torch.device('cuda', index=config.device_id)
else:
    device = torch.device('cpu')

dataset = get_dataset(dataset_root=config.datasets.dataset_root,
                      dataset_name=config.datasets.dataset_name)

dataloader_params = {'batch_size': len(dataset),
                     'stratified': config.stratified,
                     'random_split_flag': config.datasets.random_split_flag,
                     'data_split_ratio': config.datasets.data_split_ratio,
                     'seed': config.datasets.seed}
dataloader = get_dataloader(dataset, **dataloader_params)

y_true = dataset.data.y.numpy()
smiles = dataset.data.smiles

In [4]:
if config.datasets.dataset_name == 'tox21':
    for y in y_true:
        for i,elem in enumerate(y):
            if elem != 0 and elem != 1:
                y[i] = 0.

In [5]:
fingerprints = []
for smile in smiles:
    mol = Chem.MolFromSmiles(smile)
    fingerprint = np.array(Chem.RDKFingerprint(mol))
    fingerprints.append(fingerprint)
    
fingerprints = np.array(fingerprints)

RDKit WARNING: [15:41:17] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [15:41:17] WARNING: not removing hydrogen atom without neighbors
[15:41:17] WARNING: not removing hydrogen atom without neighbors
[15:41:17] WARNING: not removing hydrogen atom without neighbors


In [6]:
print(config.datasets.dataset_name)

hiv


## Random forest

In [8]:
accs, aucs = [], []
for run in range(15):
    x_train, x_test, y_train, y_test = train_test_split(fingerprints, y_true, test_size=0.2, random_state=123)
    x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=123)
    
    if config.datasets.dataset_name == 'bace' or config.datasets.dataset_name == 'hiv':
        y_train = y_train.ravel()
    
    n_classes = 2
    if config.datasets.dataset_name == 'tox21':
        n_classes = 12

    rf_model = RandomForestClassifier(n_estimators=100)
    rf_model.fit(x_train,y_train)
    
    y_pred = rf_model.predict(x_test)
    
    acc = accuracy_score(y_test,y_pred)
    auc = roc_auc_score(y_test,y_pred)
    
    accs.append(acc)
    aucs.append(auc)

In [9]:
print(f'Mean accuracy over 15 runs: {statistics.mean(accs)}')
print(f'Accuracy stdev over 15 runs: {statistics.stdev(accs)}')
print(f'Mean ROC-AUC over 15 runs: {statistics.mean(aucs)}')
print(f'ROC-AUC stdev over 15 runs: {statistics.stdev(aucs)}')

Mean accuracy over 15 runs: 0.97507091336413
Accuracy stdev over 15 runs: 0.0005654846347025965
Mean ROC-AUC over 15 runs: 0.6505712526982339
ROC-AUC stdev over 15 runs: 0.005369180440861939


## MLP

In [10]:
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
class Dataset(Dataset):
    def __init__(self, fingerprints, labels):
        self.fingerprints = fingerprints
        self.labels = labels
    def __len__(self):
        return self.fingerprints.shape[0]
    def __getitem__(self, ind):
        x = self.fingerprints[ind]
        y = self.labels[ind]
        return x, y
    
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(2048, 256)
        self.l2 = nn.Linear(256, 128)
        self.l3 = nn.Linear(128, 64)
        self.l4 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        
        if config.datasets.dataset_name == 'tox21':
            self.l5 = nn.Linear(32, 12)
            self.act = nn.Softmax(dim=1)
        else:
            self.l5 = nn.Linear(32, 1)
            self.act = nn.Sigmoid()
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
        out = self.relu(out)
        out = self.l5(out)
        out = self.act(out)
        
        return out
    
    
def train_and_evaluate():
    model = MLP().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    if config.datasets.dataset_name == 'Tox21':
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.BCELoss()
    
    x_train, x_test, y_train, y_test = train_test_split(fingerprints, y_true, test_size=0.2, random_state=123)
    x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=123)
    
    if config.datasets.dataset_name == 'clintox':
        y_train = np.expand_dims(np.argmax(y_train, axis=1), axis=1)
        y_val = np.expand_dims(np.argmax(y_val, axis=1), axis=1)
        y_test = np.expand_dims(np.argmax(y_test, axis=1), axis=1)
        
    
    train_set = Dataset(x_train, y_train)
    val_set = Dataset(x_val, y_val)
    test_set = Dataset(x_test, y_test)
    
    batch_size = 32
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader  = DataLoader(val_set,  batch_size=batch_size, shuffle=False)
    test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=False)
    
    model.train()
    
    for epoch in range(5):
        # TRAINING
        losses, accs = [], []
        ys, outs = [], []
        
        for batch_num, input_data in enumerate(train_loader):
            
            optimizer.zero_grad()
            x, y = input_data
            x = x.to(device).float()
            y = y.to(device)

            output = model(x)
            loss = criterion(output, y.float())
            loss.backward()
            losses.append(loss.item())
            
            if config.datasets.dataset_name == 'tox21':
                accs.append(np.sum((torch.argmax(y,axis=1)==torch.argmax(torch.round(output),axis=1)).cpu().numpy())/batch_size)
            else:
                accs.append(np.sum((y==torch.round(output)).cpu().numpy())/batch_size)

            ys.extend(y.cpu().numpy())
            outs.extend(output.cpu().detach().numpy())
            
            optimizer.step()
            
        ys = np.array(ys)
        outs = np.array(outs)
        auc = roc_auc_score(ys,outs)
        
        print(f'Epoch {epoch} | Loss: {statistics.mean(losses)}, Accuracy: {statistics.mean(accs)}, ROC-AUC: {auc}')
        
        # VALIDATING
        model.eval()
        losses, accs = [], []
        ys, outs = [], []
        
        for batch_num, input_data in enumerate(val_loader):
            
            x, y = input_data
            x = x.to(device).float()
            y = y.to(device)

            output = model(x)
            loss = criterion(output, y.float())
            losses.append(loss.item())
            
            if config.datasets.dataset_name == 'tox21':
                accs.append(np.sum((torch.argmax(y,axis=1)==torch.argmax(torch.round(output),axis=1)).cpu().numpy())/batch_size)

            else:
                accs.append(np.sum((y==torch.round(output)).cpu().numpy())/batch_size)
            
            ys.extend(y.cpu().numpy())
            outs.extend(output.cpu().detach().numpy())
            
        ys = np.array(ys)
        outs = np.array(outs)
        auc = roc_auc_score(ys,outs)
            
        print(f'VALID: Epoch {epoch} | Loss: {statistics.mean(losses)}, Accuracy: {statistics.mean(accs)}, ROC-AUC: {auc}\n')
        
        
    # TESTING
    model.eval()
    losses, accs = [], []
    ys, outs = [], []

    for batch_num, input_data in enumerate(test_loader):

        x, y = input_data
        x = x.to(device).float()
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y.float())
        losses.append(loss.item())
        
        if config.datasets.dataset_name == 'tox21':
            accs.append(np.sum((torch.argmax(y,axis=1)==torch.argmax(torch.round(output),axis=1)).cpu().numpy())/batch_size)
        else:
            accs.append(np.sum((y==torch.round(output)).cpu().numpy())/batch_size)
        
        ys.extend(y.cpu().numpy())
        outs.extend(output.cpu().detach().numpy())
        
    ys = np.array(ys)
    outs = np.array(outs)
    auc = roc_auc_score(ys,outs)

        
    print(f'\nTEST: Loss: {statistics.mean(losses)}, Accuracy: {statistics.mean(accs)}, ROC-AUC: {auc}')
    
    return statistics.mean(losses), statistics.mean(accs), auc

In [12]:
losses, accs, aucs = [], [], []

for run in range(15):
    loss, acc, auc = train_and_evaluate()
    losses.append(loss)
    accs.append(acc)
    aucs.append(auc)
    
print(f'Mean accuracy over 15 runs: {statistics.mean(accs)}')
print(f'Accuracy stdev over 15 runs: {statistics.stdev(accs)}')
print(f'Mean ROC-AUC over 15 runs: {statistics.mean(aucs)}')
print(f'ROC-AUC stdev over 15 runs: {statistics.stdev(aucs)}')

Epoch 0 | Loss: 0.14449093614540184, Accuracy: 0.9637390670553936, ROC-AUC: 0.7156021730317245
VALID: Epoch 0 | Loss: 0.13284554214455013, Accuracy: 0.9607558139534884, ROC-AUC: 0.7917761531637082

Epoch 1 | Loss: 0.1172282320697972, Accuracy: 0.9661686103012633, ROC-AUC: 0.823331222466842
VALID: Epoch 1 | Loss: 0.11392585530357305, Accuracy: 0.9641472868217055, ROC-AUC: 0.8237315368518006

Epoch 2 | Loss: 0.10405188348991444, Accuracy: 0.969812925170068, ROC-AUC: 0.8641097184121297
VALID: Epoch 2 | Loss: 0.12705461794958095, Accuracy: 0.9634205426356589, ROC-AUC: 0.8025865338381001

Epoch 3 | Loss: 0.09778555947732889, Accuracy: 0.9705721574344023, ROC-AUC: 0.8887381376611685
VALID: Epoch 3 | Loss: 0.11268123867380064, Accuracy: 0.9660852713178295, ROC-AUC: 0.8291240290127929

Epoch 4 | Loss: 0.09170548527535917, Accuracy: 0.9715439747327502, ROC-AUC: 0.9118579429842494
VALID: Epoch 4 | Loss: 0.1171326726068591, Accuracy: 0.9646317829457365, ROC-AUC: 0.8378011160850337


TEST: Loss: 0

## MPNN

In [13]:
import deepchem as dc
from deepchem.models.torch_models import MPNNModel
from deepchem.molnet import load_bbbp, load_bace_classification, load_clintox, load_tox21, load_hiv

In [59]:
auc = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
acc = dc.metrics.Metric(dc.metrics.accuracy_score, np.mean)

accs, aucs = [], []

for i in range(15):
    tasks, datasets, transformers = load_bbbp(featurizer=dc.feat.MolGraphConvFeaturizer(use_edges=True), splitter='random')        
    train_dataset, valid_dataset, test_dataset = datasets

    model = MPNNModel(mode='classification', n_tasks=1, batch_size=16, learning_rate=0.001)
    
    model.fit(train_dataset, nb_epoch=5)
    
    out = model.evaluate(test_dataset, [acc,auc], transformers)

    accs.append(out['mean-accuracy_score'])
    aucs.append(out['mean-roc_auc_score'])

In [60]:
print(f'Mean accuracy over 15 runs: {statistics.mean(accs)}')
print(f'Accuracy stdev over 15 runs: {statistics.stdev(accs)}')
print(f'Mean ROC-AUC over 15 runs: {statistics.mean(aucs)}')
print(f'ROC-AUC stdev over 15 runs: {statistics.stdev(aucs)}')

Mean accuracy over 15 runs: 0.8718954248366013
Accuracy stdev over 15 runs: 0.01950838454173687
Mean ROC-AUC over 15 runs: 0.8800362531844014
ROC-AUC stdev over 15 runs: 0.01639792833703606


In [61]:
auc = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
acc = dc.metrics.Metric(dc.metrics.accuracy_score, np.mean)

accs, aucs = [], []

for i in range(15):
    tasks, datasets, transformers = load_bace_classification(featurizer=dc.feat.MolGraphConvFeaturizer(use_edges=True), splitter='random')        
    train_dataset, valid_dataset, test_dataset = datasets

    model = MPNNModel(mode='classification', n_tasks=1, batch_size=16, learning_rate=0.001)
    
    model.fit(train_dataset, nb_epoch=5)
    
    out = model.evaluate(test_dataset, [acc,auc], transformers)

    accs.append(out['mean-accuracy_score'])
    aucs.append(out['mean-roc_auc_score'])
    
print(f'Mean accuracy over 15 runs: {statistics.mean(accs)}')
print(f'Accuracy stdev over 15 runs: {statistics.stdev(accs)}')
print(f'Mean ROC-AUC over 15 runs: {statistics.mean(aucs)}')
print(f'ROC-AUC stdev over 15 runs: {statistics.stdev(aucs)}')

Mean accuracy over 15 runs: 0.6390350877192983
Accuracy stdev over 15 runs: 0.032788047535735415
Mean ROC-AUC over 15 runs: 0.6948196248196248
ROC-AUC stdev over 15 runs: 0.029353265548297893


In [9]:
auc = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
acc = dc.metrics.Metric(dc.metrics.accuracy_score, np.mean)

accs, aucs = [], []

for i in range(15):
    tasks, datasets, transformers = load_clintox(featurizer=dc.feat.MolGraphConvFeaturizer(use_edges=True), splitter='random')        
    train_dataset, valid_dataset, test_dataset = datasets
    #train_dataset = dc.data.DiskDataset.from_numpy(train_dataset.X, np.argmax(train_dataset.y, axis=1), np.argmax(train_dataset.w, axis=1))
    #test_dataset = dc.data.DiskDataset.from_numpy(test_dataset.X, np.argmax(test_dataset.y, axis=1), np.argmax(test_dataset.w, axis=1))
    
    

    model = MPNNModel(mode='classification', n_tasks=2, batch_size=32, learning_rate=0.0001)
    
    model.fit(train_dataset, nb_epoch=5)
    
    out = model.evaluate(test_dataset, [acc,auc], transformers)

    accs.append(out['mean-accuracy_score'])
    aucs.append(out['mean-roc_auc_score'])
    
print(f'Mean accuracy over 15 runs: {statistics.mean(accs)}')
print(f'Accuracy stdev over 15 runs: {statistics.stdev(accs)}')
print(f'Mean ROC-AUC over 15 runs: {statistics.mean(aucs)}')
print(f'ROC-AUC stdev over 15 runs: {statistics.stdev(aucs)}')

Mean accuracy over 15 runs: 0.5135135135135135
Accuracy stdev over 15 runs: 0.09619271924707891
Mean ROC-AUC over 15 runs: 0.7212684530544111
ROC-AUC stdev over 15 runs: 0.015228555209421749


In [7]:
auc = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
acc = dc.metrics.Metric(dc.metrics.accuracy_score, np.mean)

accs, aucs = [], []

for i in range(15):
    tasks, datasets, transformers = load_tox21(featurizer=dc.feat.MolGraphConvFeaturizer(use_edges=True), splitter='random')        
    train_dataset, valid_dataset, test_dataset = datasets

    model = MPNNModel(mode='classification', n_tasks=12, batch_size=16, learning_rate=0.001)
    
    model.fit(train_dataset, nb_epoch=5)
    
    out = model.evaluate(test_dataset, [acc,auc], transformers)

    accs.append(out['mean-accuracy_score'])
    aucs.append(out['mean-roc_auc_score'])
    
print(f'Mean accuracy over 15 runs: {statistics.mean(accs)}')
print(f'Accuracy stdev over 15 runs: {statistics.stdev(accs)}')
print(f'Mean ROC-AUC over 15 runs: {statistics.mean(aucs)}')
print(f'ROC-AUC stdev over 15 runs: {statistics.stdev(aucs)}')

Mean accuracy over 15 runs: 0.7205669224211424
Accuracy stdev over 15 runs: 0.05749578424646283
Mean ROC-AUC over 15 runs: 0.7649359333236524
ROC-AUC stdev over 15 runs: 0.00750165211677537


In [14]:
auc = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
acc = dc.metrics.Metric(dc.metrics.accuracy_score, np.mean)

accs, aucs = [], []

for i in range(15):
    tasks, datasets, transformers = load_hiv(featurizer=dc.feat.MolGraphConvFeaturizer(use_edges=True), splitter='random')        
    train_dataset, valid_dataset, test_dataset = datasets

    model = MPNNModel(mode='classification', n_tasks=1, batch_size=64, learning_rate=0.001)
    
    model.fit(train_dataset, nb_epoch=5)
    
    out = model.evaluate(test_dataset, [acc,auc], transformers)

    accs.append(out['mean-accuracy_score'])
    aucs.append(out['mean-roc_auc_score'])
    
print(f'Mean accuracy over 15 runs: {statistics.mean(accs)}')
print(f'Accuracy stdev over 15 runs: {statistics.stdev(accs)}')
print(f'Mean ROC-AUC over 15 runs: {statistics.mean(aucs)}')
print(f'ROC-AUC stdev over 15 runs: {statistics.stdev(aucs)}')

RDKit WARNING: [16:02:35] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [16:02:35] WARNING: not removing hydrogen atom without neighbors
[16:02:35] WARNING: not removing hydrogen atom without neighbors
[16:02:35] WARNING: not removing hydrogen atom without neighbors


Mean accuracy over 15 runs: 0.9652808169219548
Accuracy stdev over 15 runs: 0.0007247487297151394
Mean ROC-AUC over 15 runs: 0.7529858428282149
ROC-AUC stdev over 15 runs: 0.014268942122101167
